<a href="https://colab.research.google.com/github/sudeepj24/Image_Classifier/blob/main/Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Image classification using a convolutional neural network (CNN) in PyTorch**


In [ ]:
# Import necessary libraries and modules
from google.colab import files
import os
import zipfile
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image

In [ ]:
# Upload the assignment.zip file
uploaded = files.upload()

Saving assignment.zip to assignment.zip


In [ ]:
# Extract the uploaded assignment.zip file
for fn in uploaded.keys():
    if fn.endswith('.zip'):
        with zipfile.ZipFile(fn, 'r') as zip_ref:
            zip_ref.extractall('assignment')

In [ ]:
# Define paths to train and test folders
train_path = "/content/assignment/assignment/train"
test_path = "/content/assignment/assignment/test"

# Verify the uploaded files
print("Uploaded files:", os.listdir(train_path))

Uploaded files: ['1', '0']


In [ ]:
# Define data preprocessing transforms
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((64, 64)),  # Resize to (64, 64)
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values
])

# Load the dataset
train_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)

        # Calculate the size of the input to the fully connected layer
        self.fc_input_size = self.calculate_fc_input_size()

        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten the output tensor before passing it to the fully connected layer
        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def calculate_fc_input_size(self):
        # Dummy input to calculate the size after passing through convolutional layers
        x = torch.randn(1, 1, 64, 64)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        return x.size(1) * x.size(2) * x.size(3)

In [ ]:
# Create an instance of the model
net = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# Train the model
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.100
[1,   200] loss: 0.046
[1,   300] loss: 0.037
[2,   100] loss: 0.025
[2,   200] loss: 0.021
[2,   300] loss: 0.020
[3,   100] loss: 0.018
[3,   200] loss: 0.017
[3,   300] loss: 0.018
[4,   100] loss: 0.012
[4,   200] loss: 0.013
[4,   300] loss: 0.011
[5,   100] loss: 0.008
[5,   200] loss: 0.010
[5,   300] loss: 0.003
Finished Training


In [ ]:
# Evaluating the model using different metrics

# Define data loaders for test dataset
test_dataset = torchvision.datasets.ImageFolder(root=test_path, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Lists to store predicted and ground truth labels
predicted_labels = []
true_labels = []

# Evaluate the model on the test dataset
for data in test_loader:
    inputs, labels = data
    outputs = net(inputs)
    _, predicted = torch.max(outputs, 1)
    predicted_labels.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.891
Precision: 0.902636185768716
Recall: 0.891
F1 Score: 0.8902067437234016


In [ ]:
# Testing a sample image for classification
uploaded_image = files.upload()


# Process the uploaded image for classification
for fn in uploaded_image.keys():
    img_path = fn
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        output = net(img_tensor)
        _, predicted = torch.max(output, 1)

    print("Predicted class:", predicted.item())

Saving 0000.png to 0000 (1).png
Predicted class: 1
